In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import your CRUD Python module for MongoDB interaction
from animalShelter import AnimalShelter

# Hardcoding the username and password for MongoDB connection
username = "aacuser"
password = "SNHU1234"

# Instantiate the AnimalShelter class with only username and password
shelter = AnimalShelter(username, password)

# Retrieve all data from MongoDB using the CRUD module's read function
df = pd.DataFrame.from_records(shelter.read({}))

# Remove the '_id' column to avoid issues with Dash DataTable 
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Grazioso_Salvare_Dashboard')

# Encode and add image logo
image_filename = 'GSLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Add layout structure, including logo and unique identifier
app.layout = html.Div([
    
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'100px'}),
            href='http://www.snhu.edu', target="_blank"
        ),
        html.H1('SNHU CS-340 Dashboard', style={'textAlign': 'center', 'fontWeight': 'bold'}),
        html.H4('Created by N. Vito', style={'textAlign': 'center', 'color': 'gray'})
    ], style={'textAlign': 'center'}),
    
    html.Hr(),

    # Add interactive filter options
    html.Div([
        html.Label('Rescue Type'),
        dcc.RadioItems(
            id='rescue-type-filter',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster Rescue'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block'}
        ),
    ], style={'padding': '10px', 'fontWeight': 'bold', 'fontSize': '18px', 'textAlign': 'center'}),

    html.Hr(),
    
    # Add the DataTable
    html.Div([
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            data=df.to_dict('records'),
            editable=True,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            row_selectable="single",
            selected_rows=[0],
            page_action="native",
            page_current=0,
            page_size=10,
            style_table={'height': '300px', 'overflowY': 'auto'},
            style_header={
                'backgroundColor': 'rgb(30, 30, 30)',
                'color': 'white',
                'fontWeight': 'bold'
            },
            style_cell={
                'textAlign': 'left',
                'whiteSpace': 'normal',
                'height': 'auto'
            }
        ),
    ], style={'padding': '10px'}),
    
    html.Hr(),

    # Add the chart and map side by side
    html.Div(
        className='row',
        style={'display': 'flex', 'padding': '10px'},
        children=[
            # Add the graph placeholder for charts
            html.Div(
                id='graph-id',
                className='col s12 m6',
                style={'width': '50%'},
            ),
            
            # Add the map placeholder
            html.Div(
                id='map-id',
                className='col s12 m6',
                style={'width': '50%'},
            ),
        ]
    ),
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to filter the data based on rescue type and animal type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-filter', 'value')]
)
def update_dashboard(rescue_type):
    query = {}

    if rescue_type == 'Water Rescue':
        query = {
            "animal_type": "Dog", 
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, 
            "sex_upon_outcome": "Intact Female", 
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == 'Mountain Rescue':
        query = {
            "animal_type": "Dog", 
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, 
            "sex_upon_outcome": "Intact Male", 
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == 'Disaster Rescue':
        query = {
            "animal_type": "Dog", 
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, 
            "sex_upon_outcome": "Intact Male", 
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    elif rescue_type == 'reset':
        query = {}

    # Query the MongoDB database and create a DataFrame from the results
    filtered_data = pd.DataFrame.from_records(shelter.read(query))

    # Debugging: Print the breeds available in the filtered data
    print("Filtered Breeds:", filtered_data['breed'].unique())

    if '_id' in filtered_data.columns:
        filtered_data.drop(columns=['_id'], inplace=True)
    
    return filtered_data.to_dict('records')

# Callback to update the chart visualization based on filtered data
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available")

    dff = pd.DataFrame.from_dict(viewData)

    if 'breed' in dff.columns:
        # Debugging: Print the breeds available in the pie chart data
        print("Pie Chart Breeds:", dff['breed'].unique())

        # Generate pie chart with percentage labels inside the pie chart
        fig = px.pie(
            dff,
            names='breed',
            title='Animal Breeds Distribution',
            hole=0.3, 
        )
        
        # Position percentages inside the pie chart and configure text size
        fig.update_traces(
            textinfo='percent+label',  # Show percentage and breed name
            textposition='inside',     
            textfont_size=14           
        )
        
        # Adjust the layout of the pie chart to make it large enough
        fig.update_layout(
            height=600,  
            width=600,   
            margin=dict(l=40, r=40, t=40, b=40),  # Set margins to prevent overlap
            showlegend=True
        )

        return dcc.Graph(figure=fig)
    else:
        return html.Div("Breed data not available")

# Callback to highlight selected rows in the DataTable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # If no columns are selected, return an empty style list
    if selected_columns is None or len(selected_columns) == 0:
        return []

    # Return a list of style dictionaries for selected columns
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Callback to update the geolocation map based on the selected row in the DataTable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available for the map")

    dff = pd.DataFrame.from_dict(viewData)
    
    # Check if rows are selected, if not default to the first row
    if selected_rows is None or len(selected_rows) == 0:
        row = 0
    else:
        row = selected_rows[0]

    lat = dff.iloc[row].get('location_lat')
    long = dff.iloc[row].get('location_long')

    if pd.notna(lat) and pd.notna(long):
        return [
            dl.Map(
                style={'width': '1000px', 'height': '500px'},
                center=[lat, long],
                zoom=10,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(
                        position=[lat, long],
                        children=[
                            dl.Tooltip(dff.iloc[row, 4]),  # Breed information
                            dl.Popup([
                                html.H1("Animal Name"),
                                html.P(dff.iloc[row, 9])  # Animal name
                            ])
                        ]
                    )
                ]
            )
        ]
    else:
        return html.Div("Location data not available")


# Run the server
if __name__ == '__main__':
    app.run_server(debug=True)

Connection Successful
Dash app running on http://127.0.0.1:26049/
